# Michelle's Data Capstone Project - Week 3

Peer Graded Review - Part 2

First section is just duplicating the work done in part 1 in separate notebook

In [17]:
import numpy as np 
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
import re

sourcepage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(sourcepage, "lxml")

right_table=soup.find('table', class_='wikitable sortable')

P=[]
B=[]
N=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        P.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        N.append(cells[2].find(text=True))
        
df_codes = pd.DataFrame(data=[P, B, N]).transpose()
df_codes.columns = ['Postal Code', 'Borough', 'Neighborhood']

df_codes.drop(df_codes[df_codes['Borough']=="Not assigned"].index, axis=0, inplace=True)
df1 = df_codes.reset_index()

df_group=df1.groupby('Postal Code').agg(lambda x: ','.join(x))
df_group.loc[df_group['Neighborhood']=="Not assigned",'Neighborhood']=df_group.loc[df_group['Neighborhood']=="Not assigned",'Borough']

df2 = df_group.reset_index()
df2['Borough']= df2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df2.head(10)

Postal Code      Borough                                     Neighborhood
0         M1B  Scarborough                                    Rouge,Malvern
1         M1C  Scarborough             Highland Creek,Rouge Hill,Port Union
2         M1E  Scarborough                Guildwood\n,Morningside,West Hill
3         M1G  Scarborough                                           Woburn
4         M1H  Scarborough                                      Cedarbrae\n
5         M1J  Scarborough                              Scarborough Village
6         M1K  Scarborough      East Birchmount Park\n,Ionview,Kennedy Park
7         M1L  Scarborough                    Clairlea,Golden Mile,Oakridge
8         M1M  Scarborough  Cliffcrest,Cliffside,Scarborough Village West\n
9         M1N  Scarborough                     Birch Cliff,Cliffside West\n

After issues using geocoding, decided to import via the provided CSV file

In [18]:
df3 = pd.read_csv(r'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
df3.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [19]:
df3.set_index("Postal Code")
df2.set_index("Postal Code")
postal_data=pd.merge(df2, df3)
postal_data.head(10)

Postal Code      Borough                                     Neighborhood  \
0         M1B  Scarborough                                    Rouge,Malvern   
1         M1C  Scarborough             Highland Creek,Rouge Hill,Port Union   
2         M1E  Scarborough                Guildwood\n,Morningside,West Hill   
3         M1G  Scarborough                                           Woburn   
4         M1H  Scarborough                                      Cedarbrae\n   
5         M1J  Scarborough                              Scarborough Village   
6         M1K  Scarborough      East Birchmount Park\n,Ionview,Kennedy Park   
7         M1L  Scarborough                    Clairlea,Golden Mile,Oakridge   
8         M1M  Scarborough  Cliffcrest,Cliffside,Scarborough Village West\n   
9         M1N  Scarborough                     Birch Cliff,Cliffside West\n   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  
5  43.744734 -79.239476  
6  43.727929 -79.262029  
7  43.711112 -79.284577  
8  43.716316 -79.239476  
9  43.692657 -79.264848

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

Assign the values in each row (designated by td), to the respective columns (P, B or N designated by tr)

Convert the data from the rows and columns to a dataframe output

Postal Code           Borough      Neighborhood
0         M1A      Not assigned     Not assigned

1         M2A      Not assigned     Not assigned

2         M3A        North York         Parkwoods
3         M4A        North York  Victoria Village
4         M5A  Downtown Toronto      Harbourfront

Since the class assignment requested: "Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned", we have to filter out all rows with no Borough and reset the indices so that the count starts back at 0.

index Postal Code           Borough      Neighborhood
0       2         M3A        North York         Parkwoods
1       3         M4A        North York  Victoria Village
2       4         M5A  Downtown Toronto      Harbourfront
3       5         M6A        North York  Lawrence Heights
4       6         M6A        North York    Lawrence Manor
5       7         M7A  Downtown Toronto      Queen's Park
6       9         M9A         Etobicoke  Islington Avenue
7      10         M1B       Scarborough             Rouge
8      11         M1B       Scarborough           Malvern
9      13         M3B        North York  Don Mills North

10     14         M4B         East York  Woodbine Gardens
11     15         M4B         East York     Parkview Hill
12     16         M5B  Downtown Toronto          Ryerson

13     17         M5B  Downtown Toronto  Garden District

14     18         M6B        North York        Glencairn

15     21         M9B         Etobicoke       Cloverdale

16     22         M9B         Etobicoke         Islington
17     23         M9B         Etobicoke     Martin Grove

18     24         M9B         Etobicoke  Princess Gardens
19     25         M9B         Etobicoke   West Deane Park

Next, the assignment asked for: "If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough".  So we set the Neighborhood to the same as Borough, then reset the index again.

Borough  \
Postal Code                                        
M1B                      Scarborough,Scarborough   
M1C          Scarborough,Scarborough,Scarborough   
M1E          Scarborough,Scarborough,Scarborough   
M1G                                  Scarborough   
M1H                                  Scarborough   

                                     Neighborhood  
Postal Code                                        
M1B                                 Rouge,Malvern  
M1C          Highland Creek,Rouge Hill,Port Union  
M1E             Guildwood\n,Morningside,West Hill  
M1G                                        Woburn  
M1H                                   Cedarbrae\n

Postal Code                              Borough  \
0         M1B              Scarborough,Scarborough   
1         M1C  Scarborough,Scarborough,Scarborough   
2         M1E  Scarborough,Scarborough,Scarborough   
3         M1G                          Scarborough   
4         M1H                          Scarborough   

                           Neighborhood  
0                         Rouge,Malvern  
1  Highland Creek,Rouge Hill,Port Union  
2     Guildwood\n,Morningside,West Hill  
3                                Woburn  
4                           Cedarbrae\n

Now we clean up the data so that we don't have multiples of the same Borough showing in the cell.

Postal Code      Borough                          Neighborhood
0         M1B  Scarborough                         Rouge,Malvern
1         M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2         M1E  Scarborough     Guildwood\n,Morningside,West Hill
3         M1G  Scarborough                                Woburn
4         M1H  Scarborough                           Cedarbrae\n

Final request: In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

(103, 3)